# Programmability

In [6]:
import random
random.seed(1234)

In [7]:
#let's create some dummy random string to represent as addresses
addresses = [hex(random.randint(0,2**32-1)) for _ in range(10)]

def randhex(nbits):
    return hex(random.randint(0,2**nbits-1))

## The UXTO model

We said that transactions are something like `{from: '0xabcde', to: '0xfdea',
amount: 10, signature: '0x9aa0bc123'}`, and that the state is, at any moment, a
balance table like:

|address|balance|
|-------|-------|
|`{{addresses[0]}}`|100    |
|`{{addresses[1]}}`|200|
|...|...|

In Bitcoin things are a bit different. First of all, the _state_ of the system is not a balance table, but rather a set of _unspent transaction outputs_, or _UTXOs_.

What are them? We can imagine them as bunches of coins, that belong to a certain address/public key.

|utxo id| amount| owner|
|-------|-------|------|
|{{ randhex(32)}}| {{round(random.random()*100, 4)}}| {{randhex(32)}}|
|{{ randhex(32)}}| {{round(random.random()*100, 4)}}| {{randhex(32)}}|
|{{ randhex(32)}}| {{round(random.random()*100, 4)}}| {{randhex(32)}}|

Now, each of these transactions can be "spent", that is their ownership can be transfered, by their current owner. But what if I own a transaction worth 10, and want to just spend 5? The answer is that the mechanism allows me to take a transaction, split it arbitrarily in n _output transactions_, and then transfer ownership to different accounts. Then I can take my 10 bitcoins, split it into 2 outputs (X and Y) worth 5 each, and transfer X to the recipient, and send Y back to me. 
And what if I own (I control) two different transactions worth 5 each, and I want to transfer 10 to a recipient? Well, I can join different transactions into one, and then transfer is to the recipient.
So, to sum up: a transaction can take a set of unspent transactions as _input_, merge them into one, and then split it into outputs, that can be distributed to different owners. (See for instance https://www.blockchain.com/btc/tx/c0a527beb446f09d4363381a38da288dbc41db2911daa84617ae0b2947fb1b9a).


## Bitcoin script

But things are more general than that. Control of a UTXO is not expressed as a public key, but rather as a "coding puzzle", or "locking script". There's a non Turing complete, stack based script language, called _Bitcoin script_, in which these conditions are expressed.
The idea is this: a locking script is a piece of Bitcoin script code. To unlock the transaction, you have to provide another piece of code that prepended to the locking script builds a script that returns a value of 1.
Let's see with an example:
`3 OP_ADD 5 OP_EQUAL`
In Bitcoin script this means:
* add 3 on top of the stack ("naked" numbers here means pushing on top of the stack)
* sum the two elements on top of the stack (pops the two elements, and push the result)
* push 5 on top of the stack
* compare the two elements on top of the stack, and if they're equal, push 1, otherwise push 0

So, what is the unlocking script here? But `2`, of course.
When you want to spend a transaction, you have to provide the prepending script that unlocks it. Then you have the right to choose the new locking script for the outputs.

Locking a transaction to make it redeemable by the owner of a public/private key is a special case here:

`OP_DUP OP_HASH160 <Bob Public Key Hash> OP_EQUALVERIFY OP_CHECKSIG`

(explanation of the opcodes)

Here the unlocking script is:

`<Bob Signature> <Bob Public Key>`

But it is possible to do many other things with this kind of programmability!

* Complex spending conditions, such as "this transaction can be spent only by Bob or, after 1 month has elapsed, by either Bob or Alice and Carol together"
* we can create offline channels and structures (like in the Lightning network)



## Limits of Bitcoin script

There are limits to what it is possible to do with Bitcoin script

* it is not Turing complete: we don't have loops, so for instance we can iterate over data structures such as lists. We can unroll loops, to a certain extent, but it gets difficult fast
*  it's stateless: we cannot create spending conditions that require multiple steps, or protocols. For example, imagine that you want to pay either Alice or Bob depending on who wins ten rounds of rock paper scissors. Or you cannot easily allow for commit-reveal protocols, needed for example to secure bounties. 
* value blindness: it's not possible to allow partial spending of a transaction according to unlocking script. Example: Bob puts 3000 euros into a transaction, and pay 1000 euro for every root of a given 3rd grade polynomial provided. Either the transaction is spent, or it is not
* the transactions cannot access blockchain data, and is thus very difficult to provide them with credible sources of randomness (for istance for a lottery)

To overcome these limitations, Ethereum was invented (https://ethereum.org/en/whitepaper/)




## Ethereum

Ethereum borns as a blockchain based accounting systems like Bitcoin, with some differences.

1. it is not based on UTXO's, but rather on actual accounts/balance
2. the PoW mechanism is different, and tries to prevent ASIC accelerability
3. it features a Turing complete scripting language

## Things to say (programmability in Ethereum?)

* Account model
* There are two kinds of accounts in Ethereum: _externally owned_, and _contracts_
* Externally owned are accounts associated with a public key. Their state is given by _balance_ in ether, and by the _nonce_, that is the number of transaction registered so far for that account
* Externally owned account state can be modified by transactions signed with the corresponding private key, transferring ethers
* Contract accounts state also contains _code_ and _storage_
* It is possible to send a transaction addressed to a contract account containing _data_. The code associated with the contract will be executed, and the transaction data (calldata) will be fed as an input
* contracts can of course receive ether
* The code of a contract can:
    * read the transaction calldata
    * transfer money to another address
    * read/write its storage
    
https://ethereum.org/en/whitepaper/

What happens if some smart contract code goes into an infinite loop? And what about the cost that miners have to pay to compute state evolution?
In Ethereum, the computation required by a transaction is given a cost in unit of measurement called _gas_. Each EVM instruction has a gas cost. There is a maximum gas limit that can be spent for a transaction.
The user that submit a transaction declares how much he is willing to spend for each gas unit. The total fees are `gas_price*gas_consumed`. Of course, miners will prefer transactions with higher gas price.
    
## Applications

### Tokens

Let's go back to our babysitting problem. Can we tackle it with our new tools?
Now we have an accounting system that works for native tokens, bitcoin and ether. But here we would like to create our own token, with its peculiar properties.
Well, we can create a smart contract, whose state is the account balance table!

```
pragma solidity >= 0.6.0;

contract Token {
    
    mapping(address => uint256) public balances;
    
    constructor() {
        balances[msg.sender] = 100;
    }
    
    function transfer(address _to, uint256 _value) public {
        require(balances[msg.sender] >= _value);
        balances[msg.sender] -= _value;
        balances[_to] += _value;
    }
    
}
```

Questions:
1. how many tokens are there? 
2. Would a montary manoveur like that of the babysitting coop be possible here?
3. What other "monetary policies" can we imagine? How can we implement them?

Since this is a very foundational application, a standard was defined and widely adopted: ERC20.

https://ethereum.org/en/developers/docs/standards/tokens/erc-20/

### Saving accounts

### Insurances


### Decentarlised Autonomous Organizations

### Digital Identity

## Enabled paradigms/philosophy/software engineering

* Common knowledge computing
* Model-View-Controller -> Client-Blockchain-Client
* Blockchains enable global untamperable state



